# Merge permno_cusip_ticker_permco and crsp_comp_bdx to get gvkey in it

In [38]:
import pandas as pd

In [39]:
# read the xlsx file
gvkey = pd.read_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/CRSP_COMP_BDX_GVKEY.csv')
permco = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/PERMNO_CUSIP_TICKER_PERMCO.csv")

In [40]:
print("gvkey shape: ",gvkey.shape)
print("permco shape: ",permco.shape)

gvkey shape:  (11843, 6)
permco shape:  (187338, 18)


In [41]:
print("gvkey columns: ",gvkey.columns)
print("permco columns: ",permco.columns)

gvkey columns:  Index(['COMPANYID', 'PERMCO', 'GVKEY', 'SCORE', 'PREFERRED', 'DUPLICATE'], dtype='object')
permco columns:  Index(['PERMNO', 'FSYM_ID', 'FSYM_ID_KIND', 'PROPER_NAME', 'FSYM_REGIONAL_ID',
       'FSYM_SECURITY_ID', 'FS_PERM_SEC_ID', 'FACTSET_ENTITY_ID',
       'ENTITY_PROPER_NAME', 'CUSIP_FS', 'TICKER_EXCHANGE', 'PERMCO', 'CUSIP',
       'NCUSIP', 'TICKER', 'COMNAM', 'LINK_BDATE', 'LINK_EDATE'],
      dtype='object')


In [42]:
# Step 0: 篩選欄位 & 移除重複列
permco_less = permco[[
    'PERMNO', 'PROPER_NAME', 'ENTITY_PROPER_NAME', 'CUSIP_FS',
    'TICKER_EXCHANGE', 'PERMCO', 'CUSIP', 'NCUSIP', 'TICKER',
    'COMNAM', 'LINK_BDATE', 'LINK_EDATE'
]].copy()

permco_less.drop_duplicates(inplace=True)

# Step 1: 確保 LINK_EDATE 是數值型別（避免 NaN 比較出錯）
permco_less['LINK_EDATE'] = pd.to_numeric(permco_less['LINK_EDATE'], errors='coerce')

# Step 2: 根據 PERMNO 保留 LINK_EDATE 最大的那一筆
latest_permco = permco_less.loc[
    permco_less.groupby('PERMNO')['LINK_EDATE'].idxmax()
].reset_index(drop=True)

In [44]:
# 確保是字串格式
latest_permco['TICKER_EXCHANGE'] = latest_permco['TICKER_EXCHANGE'].astype(str)

# 拆分欄位
latest_permco[['TICKER_SYMBOL', 'EXCHANGE']] = latest_permco['TICKER_EXCHANGE'].str.split('-', expand=True)
latest_permco = latest_permco[latest_permco['EXCHANGE'].isin(['NAS', 'NYS'])].reset_index(drop=True)
latest_permco.drop(columns=['TICKER_EXCHANGE', 'TICKER_SYMBOL', 'LINK_BDATE',	'LINK_EDATE'], inplace=True)
latest_permco

,PERMNO,PROPER_NAME,ENTITY_PROPER_NAME,CUSIP_FS,PERMCO,CUSIP,NCUSIP,TICKER,COMNAM,EXCHANGE
0,10002,BancTrust Financial Group Inc.,"BancTrust Financial Group, Inc.",05978R107,7954,05978R10,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,NAS
1,10003,Great Ctry Bank Ansonia Ct,NaN,390318103,7957,39031810,39031810,GCBK,GREAT COUNTRY BK ASONIA CT,NAS
2,10006,Acf Industries,NaN,000800102,22156,00080010,00080010,ACF,A C F INDUSTRIES INC,NYS
3,10009,Iroquois Bancorp Inc.,"Iroquois Bancorp, Inc. (New York)",463347104,7965,46334710,46334710,IROQ,IROQUOIS BANCORP INC,NAS
4,10010,Cabot Medical Corporation,Cabot Medical Corp.,127095107,7967,12709510,12709510,CBOT,CABOT MEDICAL CORP,NAS
...,...,...,...,...,...,...,...,...,...,...
16293,93428,"BroadSoft, Inc.","BroadSoft, Inc.",11133B409,53446,11133B40,11133B40,BSFT,BROADSOFT INC,NAS
16294,93429,Cboe Global Markets Inc,"Cboe Global Markets, Inc.",12503M108,53447,12503M10,12503M10,CBOE,C B O E GLOBAL MARKETS INC,NAS
16295,93431,Pactera Technology International Ltd. Sponsore...,Pactera Technology International Ltd.,695255109,53449,69525510,69525510,PACT,PACTERA TECHNOLOGY INTL LTD,NAS
16296,93434,S&W Seed Company,S&W Seed Co.,785135104,53427,78513510,78513510,SANW,S & W SEED CO,NAS


In [45]:
permco_gvkey = pd.merge(latest_permco, gvkey[['COMPANYID', 'PERMCO', 'GVKEY']], how='left', on='PERMCO')
print("permco_gvkey shape: ",permco_gvkey.shape)

permco_gvkey shape:  (16722, 12)


In [46]:
permco_gvkey_without_na = permco_gvkey.dropna(subset=['GVKEY'])
print("permco_gvkey_without_na shape: ",permco_gvkey_without_na.shape)

permco_gvkey_without_na shape:  (7669, 12)


In [47]:
permco_gvkey_without_na.to_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/permno_cusip_ticker_permco_gvkey_without_na.csv', index=False)

## Merge green patent count and forward citation

In [ ]:
import pandas as pd

In [ ]:
forward = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/total_5yr_forward_citations_current.csv")
count = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_count.csv")

In [ ]:
print("forward.columns: ",forward.columns)
print("count.columns: ",count.columns)

forward.columns:  Index(['gvkey_numeric', 'filing_year', 'total_5yr_forward_citations',
       'within_5_years'],
      dtype='object')
count.columns:  Index(['gvkey_numeric', 'filing_year', 'patents_count'], dtype='object')


In [ ]:
forward_count = pd.merge(forward, count, how='left', on=['gvkey_numeric', 'filing_year'])
forward_count.to_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/total_5yr_forward_citations_v2_green_current_count.csv', index=False)

## Merge to get ticker

In [ ]:
import pandas as pd

In [ ]:
forward_count = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count.csv")
mapping = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_cusip_ticker_permco_gvkey_without_na.csv")

In [ ]:
print(forward_count.columns)
print(mapping.columns)

Index(['gvkey_numeric', 'filing_year', 'total_5yr_forward_citations',
       'within_5_years', 'patents_count'],
      dtype='object')
Index(['PERMNO', 'FSYM_ID', 'FSYM_ID_KIND', 'PROPER_NAME', 'FSYM_REGIONAL_ID',
       'FSYM_SECURITY_ID', 'FS_PERM_SEC_ID', 'FACTSET_ENTITY_ID',
       'ENTITY_PROPER_NAME', 'CUSIP_FS', 'TICKER_EXCHANGE', 'PERMCO', 'CUSIP',
       'NCUSIP', 'TICKER', 'COMNAM', 'LINK_BDATE', 'LINK_EDATE', 'COMPANYID',
       'GVKEY'],
      dtype='object')


In [ ]:
forward_count_ticker = pd.merge(forward_count, mapping[['GVKEY', 'PERMNO', 'CUSIP', 'TICKER']], how='left', left_on='gvkey_numeric', right_on='GVKEY')

In [ ]:
forward_count_ticker_dropna = forward_count_ticker.dropna()
forward_count_ticker_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker.csv", index=False)

In [ ]:
report_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/company_name/company_name_ticker.csv")

In [ ]:
forward_count_ticker_dropna = forward_count_ticker_dropna[forward_count_ticker_dropna['TICKER'].isin(report_ticker['ticker'].values)]
forward_count_ticker_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_within_report.csv", index=False)

# Merge the ticker in mapping to gp_values by permno

In [2]:
import pandas as pd

In [3]:
gp_value = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_values_filing.csv")
mapping = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_cusip_ticker_permco_gvkey_without_na.csv")

In [4]:
print(gp_value.columns)
print(mapping.columns)

Index(['permno', 'year', 'total_values_nominal', 'patents_count_nominal',
       'total_values_real', 'patents_count_real', 'value_per_patent_nominal',
       'value_per_patent_real'],
      dtype='object')
Index(['PERMNO', 'PROPER_NAME', 'ENTITY_PROPER_NAME', 'CUSIP_FS', 'PERMCO',
       'CUSIP', 'NCUSIP', 'TICKER', 'COMNAM', 'EXCHANGE', 'COMPANYID',
       'GVKEY'],
      dtype='object')


In [6]:
gp_value_ticker = pd.merge(gp_value, mapping[['GVKEY', 'PERMNO', 'CUSIP', 'TICKER','COMNAM','EXCHANGE']], how='left', left_on='permno', right_on='PERMNO')

In [16]:
gp_value_ticker = gp_value_ticker.dropna()
gp_value_ticker.drop_duplicates(inplace=True)
gp_value_ticker.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patents_values_filing_ticker.csv", index=False)